In [ ]:
%reset

In [ ]:
import optuna
import pandas as pd
import numpy as np

from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import  mean_squared_error
from scipy import stats

import warnings
warnings.filterwarnings('ignore')

In [ ]:
#Set this value to true if hyperparameter tuning is complete and the test set should be loaded and predicted on
OUTPUT_TEST = False

In [ ]:
#Load the training and validation datasets
X_train = pd.read_csv("../data/cleaned/training.csv")
y_train = pd.read_csv("../data/cleaned/training_labels.csv")
X_val = pd.read_csv("../data/cleaned/validation.csv")
y_val = pd.read_csv("../data/cleaned/validation_labels.csv")

In [ ]:
#Some columns headers contain '[' or ']' which are not compatable with sklearn. They are change to '(' and ')' respectively.
columns = X_train.columns
for col in columns:
    if '[' in col or ']' in col:
        old_name = col
        col = col.replace('[', '(')
        col = col.replace(']', ')')
        
        X_train = X_train.rename(columns={old_name:col})
        X_val = X_val.rename(columns={old_name:col})

In [ ]:
#Splitting of the training set into a vedrification and training set with a 90/10 split. This verification set is used for optuna hyperparameter tuning.
X_train, X_verif, y_train, y_verif = train_test_split(X_train, y_train, test_size=0.1, random_state=42)

In [ ]:
#Reset the indicies after splitting the dataset
X_train = X_train.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)
X_verif = X_verif.reset_index(drop=True)
y_verif = y_verif.reset_index(drop=True)
X_val = X_val.reset_index(drop=True)
y_val = y_val.reset_index(drop=True)

In [ ]:
#Check performance with no tuning to ensure performance is improving
sanity_check = XGBRegressor()
sanity_check.fit(X_train, y_train)
val_pred = sanity_check.predict(X_val)
verif_pred = sanity_check.predict(X_verif)
sanity_verif_error = mean_squared_error(y_verif,verif_pred,squared=False)
sanity_val_error = mean_squared_error(y_val,val_pred,squared=False)
print("SANITY CHECK VALUES:")
print("Verification RMSE:", sanity_verif_error)
print("Validation RMSE:", sanity_val_error)


In [ ]:
def objective(trial):
    #Define the objective function

    params = {
        'max_depth': trial.suggest_int('max_depth', 1, 9),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 1.0),
        'n_estimators': trial.suggest_int('n_estimators', 50, 1000),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'gamma': trial.suggest_loguniform('gamma', 1e-8, 1.0),
        'subsample': trial.suggest_loguniform('subsample', 0.01, 1.0),
        'colsample_bytree': trial.suggest_loguniform('colsample_bytree', 0.01, 1.0),
        'reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-8, 1.0),
        'reg_lambda': trial.suggest_loguniform('reg_lambda', 1e-8, 1.0),
    }

    params["tree_method"] = "hist"

    # Fit the model
    optuna_model = XGBRegressor(**params)
    optuna_model.fit(X_train, y_train)

    # Make predictions
    val_pred = optuna_model.predict(X_val)
    train_pred = optuna_model.predict(X_train)
    val_loss = mean_squared_error(y_val,val_pred,squared=False)
    train_loss = mean_squared_error(y_train,train_pred,squared=False)

    # Evaluate predictions
    error = (val_loss - train_loss) + 2 * train_loss
    
    return error


In [ ]:
sampler = optuna.samplers.CmaEsSampler()
study = optuna.create_study(sampler=sampler)
study.optimize(objective, n_trials=100, timeout=1800)

print("Best trial:")
trial = study.best_trial

print("  Value: ", trial.value)

print("  Params: ")
for key, value in trial.params.items():
        print("    {}: {}".format(key, value))

In [ ]:
params = trial.params
model = XGBRegressor(**params)
print(params)
model.fit(X_train, y_train)

In [ ]:
val_pred = model.predict(X_val)
error = mean_squared_error(y_val,val_pred,squared=False)
print("RMSE:", error)
print("Difference from sanity check:", sanity_val_error - error)

In [ ]:
if not OUTPUT_TEST:
    raise ValueError("OUTPUT_TEST set to False. If you would like to output final test values set to True and continue running from here")

In [ ]:
X_test = pd.read_csv("../data/cleaned/test.csv")
y_test = pd.read_csv("../data/cleaned/test_labels.csv")

In [ ]:
columns = X_test.columns
for col in columns:
    if '[' in col or ']' in col:
        old_name = col
        col = col.replace('[', '(')
        col = col.replace(']', ')')
        
        X_test = X_test.rename(columns={old_name:col})

In [ ]:
test_preds = model.predict(X_test)
train_preds = model.predict(X_train)

In [ ]:
#Save test true vals and predictions to csv

pred_data = pd.DataFrame(test_preds)
pred_filepath = '../data/predictions/XG/test_pred_xg.csv'
pred_data.to_csv(pred_filepath, index=False, header=False)
pred_data = pd.DataFrame(y_test)
pred_filepath = '../data/predictions/XG/test_true_xg.csv'
pred_data.to_csv(pred_filepath, index=False, header=False)

#Save train true vals and predictions to csv

pred_data = pd.DataFrame(train_preds)
pred_filepath = '../data/predictions/XG/train_pred_xg.csv'
pred_data.to_csv(pred_filepath, index=False, header=False)
pred_data = pd.DataFrame(y_train)
pred_filepath = '../data/predictions/XG/train_true_xg.csv'
pred_data.to_csv(pred_filepath, index=False, header=False)

In [ ]:
#Save inputs to csv

pred_data = pd.DataFrame(X_train)
pred_filepath = '../data/predictions/XG/train_input_xg.csv'
pred_data.to_csv(pred_filepath, index=False, header=False)
true_data = pd.DataFrame(X_test)
true_filepath = '../data/predictions/XG/test_input_xg.csv'
true_data.to_csv(true_filepath, index=False, header=False)

In [ ]:
#Read in values from csv and calculate RMSE and r values

test_pred_data = np.genfromtxt('../data/predictions/XG/test_pred_xg.csv', delimiter=',', filling_values=np.nan)
test_true_data = np.genfromtxt('../data/predictions/XG/test_true_xg.csv', delimiter=',', filling_values=np.nan)
train_pred_data = np.genfromtxt('../data/predictions/XG/train_pred_xg.csv', delimiter=',', filling_values=np.nan)
train_true_data = np.genfromtxt('../data/predictions/XG/train_true_xg.csv', delimiter=',', filling_values=np.nan)

test_rmse = mean_squared_error(test_true_data,test_pred_data,squared=False)
test_r = stats.pearsonr(test_true_data,test_pred_data)

train_rmse = mean_squared_error(train_true_data,train_pred_data,squared=False)
train_r = stats.pearsonr(train_true_data,train_pred_data)

print("Train:")
print(train_rmse)
print('Test:')
print(test_rmse)
print(test_r)